# **House Price** Prediction Using Tensor Flow Decision Forest 
This notebook walks you through how to train a baseline Random Forest model using TensorFlow Decision Forests on the House Prices dataset made available for this competition.

Roughly, the code will look as follows:

```
import tensorflow_decision_forests as tfdf
import pandas as pd

dataset = pd.read_csv("project/dataset.csv")
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label="my_label")

model = tfdf.keras.RandomForestModel()
model.fit(tf_dataset)

print(model.summary())
```

Decision Forests are a family of tree-based models including Random Forests and Gradient Boosted Trees. They are the best place to start when working with tabular data, and will often outperform (or provide a strong baseline) before you begin experimenting with neural networks.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
# Removed any import or usage of numpy.strings as it does not exist.
print("All necessary libraries imported successfully.")

All necessary libraries imported successfully.


# Loading the Dataset

In this cell, we load three CSV files from the `Data` directory:

- **train.csv**: Training dataset for model development
- **test.csv**: Testing dataset for making predictions
- **sample_submission.csv**: Sample submission format for predictions

The datasets are loaded using pandas' `read_csv()` function. This will create three DataFrames:
- `train`: Contains the training data with features and target variable
- `test`: Contains the test data with features only
- `submission`: Contains the format for submitting predictions

In [27]:
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')
submission = pd.read_csv('./Data/sample_submission.csv')
print("Data loaded successfully.")
print("trainging data shape",train.shape)
print("testing data shape",test.shape)
print("submission data shape",submission.shape)

Data loaded successfully.


In [ ]:
train.head(5)